In [1]:
import bs4 as bs
import datetime as dt
import os
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import matplotlib.pyplot as plt
from matplotlib import style
import glob
%matplotlib inline
style.use('ggplot')

In [2]:
tickers = pd.read_csv('tickers_new.csv')
ticker_list = tickers['ticker'].values.tolist()
company_list = tickers['company'].values.tolist()
sector_list = tickers['sector'].values.tolist() 

In [3]:
Y = []
for symbols,company,sector in zip(ticker_list,company_list,sector_list):
    df = pd.read_csv(f'stock_csvs/stock_pup_{symbols}.csv')
    df['symbol'] = symbols
    df['company'] = company
    df['sector']  = sector
    cols = df.columns.tolist()
    cols = cols[-3:] + cols[:-3]
    df = df[cols]
    Y.append(df)

In [4]:
fund_stocks = pd.concat(Y, sort = False)

In [5]:
fund_stocks.shape

(62850, 44)

In [6]:
# Quarter End to Date time
fund_stocks['Quarter end'] = pd.to_datetime(fund_stocks['Quarter end'])
fund_stocks.set_index("Quarter end", inplace=True)

In [7]:
#2013 to current year
five_yr_fstock = fund_stocks['2000':]

In [8]:
#convert strings into numeric integers
five_yr_fstock = five_yr_fstock.apply(pd.to_numeric, errors='ignore')

In [9]:
#assign variable to unmatch
nomatch_companies = five_yr_fstock[five_yr_fstock['company'] == 'nomatch']

In [10]:
#assign variable to matched company and sector
company_fund = five_yr_fstock[five_yr_fstock['company'] != 'nomatch']

In [11]:
# 135 companies were not matched to finviz company
nomatch_companies.groupby('symbol')['company'].nunique().value_counts()

1    189
Name: company, dtype: int64

In [12]:
# 560 companies were matched to finviz company
company_fund.groupby('company')['symbol'].nunique().value_counts()

1    566
Name: symbol, dtype: int64

In [13]:
company_fund = company_fund.replace(to_replace='0', value= np.nan)

In [14]:
company_fund = company_fund.replace(to_replace='None', value= np.nan )

In [15]:
company_fund = company_fund.apply(pd.to_numeric, errors='ignore')

In [16]:
company_fund['P/E ratio'].fillna(0, inplace=True)

In [17]:
company_fund.fillna(0, inplace = True)

In [18]:
company_fund['reported_pe'] = company_fund['P/E ratio'].apply(lambda x: 1 if x != 0 else 0)

In [19]:
company_fund['reported_pe'].value_counts()

1    35178
0     5388
Name: reported_pe, dtype: int64

In [20]:
# company_fund['P/E ratio'].apply(lambda x: 1 if x >= 10 and x <= 25 else 0).value_counts()

In [22]:
company_fund

,symbol,company,sector,Shares,Shares split adjusted,Split factor,Assets,Current Assets,Liabilities,Current Liabilities,...,P/E ratio,Cumulative dividends per share,Dividend payout ratio,Long-term debt to equity ratio,Equity to assets ratio,Net margin,Asset turnover,Free cash flow per share,Current ratio,reported_pe
Quarter end,,,,,,,,,,,,,,,,,,,,,
2019-04-30,A,"Agilent Technologies, Inc.",Healthcare,315993352,315993352,1.0,9.022000e+09,3.812000e+09,3.897000e+09,1.118000e+09,...,22.16,21.390,0.1783,0.3508,0.5681,0.2226,0.58,0.67,3.4097,1
2019-01-31,A,"Agilent Technologies, Inc.",Healthcare,317515869,317515869,1.0,8.952000e+09,3.712000e+09,3.916000e+09,1.095000e+09,...,72.04,21.230,0.1711,0.3570,0.5626,0.2286,0.58,0.55,3.3900,1
2018-10-31,A,"Agilent Technologies, Inc.",Healthcare,318533054,318533054,1.0,8.541000e+09,3.848000e+09,3.970000e+09,1.171000e+09,...,73.29,21.070,0.6044,0.3939,0.5347,0.0643,0.57,1.06,3.2861,1
2018-07-31,A,"Agilent Technologies, Inc.",Healthcare,318769547,318769547,1.0,8.349000e+09,3.667000e+09,3.781000e+09,1.014000e+09,...,90.89,20.920,0.6215,0.3942,0.5467,0.0620,0.56,0.51,3.6164,1
2018-04-30,A,"Agilent Technologies, Inc.",Healthcare,319952126,319952126,1.0,8.784000e+09,4.525000e+09,4.167000e+09,1.365000e+09,...,117.27,20.770,0.7603,0.3902,0.5252,0.0502,0.55,0.80,3.3150,1
2018-01-31,A,"Agilent Technologies, Inc.",Healthcare,322476579,322476579,1.0,8.698000e+09,4.397000e+09,4.172000e+09,1.361000e+09,...,33.48,20.620,0.8925,0.3981,0.5199,0.0425,0.55,0.48,3.2307,1
2017-10-31,A,"Agilent Technologies, Inc.",Healthcare,323018027,323018027,1.0,8.426000e+09,4.169000e+09,3.591000e+09,1.263000e+09,...,32.83,20.470,0.2477,0.3728,0.5733,0.1530,0.55,0.71,3.3009,1
2017-07-31,A,"Agilent Technologies, Inc.",Healthcare,321828003,321828003,1.0,8.261000e+09,3.996000e+09,3.650000e+09,1.241000e+09,...,33.11,20.340,0.2581,0.3908,0.5578,0.1436,0.55,0.57,3.2200,1
2017-04-30,A,"Agilent Technologies, Inc.",Healthcare,321337443,321337443,1.0,8.016000e+09,3.800000e+09,3.641000e+09,1.187000e+09,...,33.70,20.210,0.2718,0.4122,0.5454,0.1341,0.55,0.67,3.2013,1


In [21]:
company['growth'] = company_fund['P/E ratio'].apply(lambda x: 1 if x == 0 or x >= 26 else 0).value_counts()

TypeError: 'str' object does not support item assignment

In [442]:
company_fund[company_fund['symbol'] == 'A'][['Price','Earnings', 'P/E ratio']]

,Price,Earnings,P/E ratio
Quarter end,,,
2019-04-30,78.02,1.820000e+08,22.16
2019-01-31,69.16,5.040000e+08,72.04
2018-10-31,66.69,1.950000e+08,73.29
2018-07-31,65.44,2.360000e+08,90.89
2018-04-30,69.19,2.050000e+08,117.27
2018-01-31,70.30,-3.200000e+08,33.48
2017-10-31,63.37,1.770000e+08,32.83
2017-07-31,58.60,1.750000e+08,33.11
2017-04-30,52.23,1.640000e+08,33.70


In [ ]:
com